In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("2019-Nov.csv")

In [1]:
# Xem parquet trước để xử lý trong spark thôi
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd

batch_dataframe = []

parquet_file = pq.ParquetFile('C:\Máy tính\BigDataProject\spark_app\data_2019-11-28_00.parquet')

for i in parquet_file.iter_batches(batch_size=350):
    batch_dataframe.append(i.to_pandas())

# Hợp nhất tất cả các DataFrame trong batch_dataframe
merged_df = pd.concat(batch_dataframe, ignore_index=True)

# In ra danh sách các cột
print("Columns in merged DataFrame:", merged_df.columns.tolist())
merged_df.info()
# In ra thông tin về DataFrame
merged_df

Columns in merged DataFrame: ['event_time', 'event_type', 'product_id', 'category_id', 'category_code', 'brand', 'price', 'user_id', 'user_session']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11304 entries, 0 to 11303
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   event_time     11304 non-null  datetime64[ns]
 1   event_type     11304 non-null  object        
 2   product_id     11304 non-null  int64         
 3   category_id    11304 non-null  int64         
 4   category_code  7681 non-null   object        
 5   brand          9916 non-null   object        
 6   price          11304 non-null  float64       
 7   user_id        11304 non-null  int64         
 8   user_session   11304 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 794.9+ KB


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-28 00:00:00,view,12300394,2053013556311359947,construction.tools.drill,None,46.82,571535080,a7d319fe-1894-4a87-8dad-0535466d9a57
1,2019-11-28 00:00:00,view,28713252,2053013565639492569,apparel.shoes,respect,89.84,554617586,2e944490-59e9-4863-90dc-412e15ca3aa1
2,2019-11-28 00:00:00,view,1004767,2053013555631882655,electronics.smartphone,samsung,246.24,515425837,aba676e8-72de-4ea2-a4da-8eb6f3dfae7e
3,2019-11-28 00:00:02,view,26300105,2053013563584283495,None,lucente,699.89,543799816,9a89c99d-eab9-40fc-842f-0ebbd7da6106
4,2019-11-28 00:00:02,view,1307353,2053013558920217191,computers.notebook,acer,411.83,568271465,f2c45be2-c4cf-4caf-9c40-eaf0634b80dc
...,...,...,...,...,...,...,...,...,...
11299,2019-11-28 00:59:59,view,1005159,2053013555631882655,electronics.smartphone,xiaomi,196.30,519813257,4fb68c73-c4ff-49e3-89ff-ff0fb0d20b94
11300,2019-11-28 00:59:59,view,4804295,2053013554658804075,electronics.audio.headphone,xiaomi,33.21,556217499,cd01e0b7-f751-4b5b-850e-01a6339b9d7a
11301,2019-11-28 00:59:59,view,1004237,2053013555631882655,electronics.smartphone,apple,1070.04,514288390,1882e979-fa9c-4b53-9d1a-b9cec3cdace3
11302,2019-11-28 00:59:59,view,6701067,2053013554247762257,computers.components.videocards,asus,507.86,569732457,8d8e89a3-819c-4adb-8989-67d05cb6910f


In [1]:
# Import các thư viện cần thiết
import pandas as pd
import pyarrow.parquet as pq
from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as F

# Đọc file Parquet và hợp nhất các batch thành DataFrame
batch_dataframe = []
parquet_file = pq.ParquetFile('C:\\Máy tính\\BigDataProject\\spark_app\\data_2019-11-28_00.parquet')

for batch in parquet_file.iter_batches(batch_size=350):
    batch_dataframe.append(batch.to_pandas())

# Hợp nhất tất cả các DataFrame trong batch_dataframe
merged_df = pd.concat(batch_dataframe, ignore_index=True)

# In danh sách các cột
print("Columns in merged DataFrame:", merged_df.columns.tolist())

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("FilterData") \
    .getOrCreate()

# Chuyển đổi Pandas DataFrame sang Spark DataFrame
spark_df = spark.createDataFrame(merged_df)

# Đổi tên cột event_time thành event_timestamp nếu tồn tại
if "event_time" in spark_df.columns:
    spark_df = spark_df.withColumnRenamed("event_time", "event_timestamp")

# Chuyển đổi event_timestamp sang kiểu timestamp và xử lý giá trị không hợp lệ
spark_df = spark_df.withColumn(
    "event_timestamp",
    F.when(
        F.col("event_timestamp").contains("UTC"),
        F.to_timestamp(
            F.regexp_replace(F.col("event_timestamp"), " UTC$", ""),
            "yyyy-MM-dd HH:mm:ss"
        )
    ).otherwise(
        F.to_timestamp(F.col("event_timestamp"), "yyyy-MM-dd HH:mm:ss")
    )
).filter(F.col("event_timestamp").isNotNull())

# Tính số lượng hoạt động trong mỗi phiên người dùng (activity_count)
window_spec = Window.partitionBy("user_session")
spark_df = spark_df.withColumn("activity_count", F.count("*").over(window_spec))

# Tách category_code thành hai lớp: category_code_level1 và category_code_level2
spark_df = spark_df.withColumn(
    "category_code_level1",
    F.regexp_replace(
        F.split(F.col("category_code"), "\\.", 2).getItem(0), "null", ""
    )
).withColumn(
    "category_code_level2",
    F.regexp_replace(
        F.split(F.col("category_code"), "\\.", 2).getItem(1), "null", ""
    )
)

# Tính ngày trong tuần từ event_timestamp
spark_df = spark_df.withColumn(
    "event_weekday", F.dayofweek(F.col("event_timestamp")) - 1
)

# Tạo cột is_purchased (giả sử event_type = "purchase" là giao dịch mua)
spark_df = spark_df.withColumn(
    "is_purchased",
    F.when(F.lower(F.col("event_type")) == "purchase", 1).otherwise(0)
)

# Điền giá trị NULL bằng giá trị mặc định
spark_df = spark_df.fillna(
    {"price": 0.0, "brand": "", "category_code": "", "event_type": ""}
)

# Lọc các bản ghi không hợp lệ
spark_df = spark_df.filter(
    F.col("user_id").isNotNull() &
    F.col("product_id").isNotNull() &
    F.col("price").isNotNull() &
    (F.col("price") >= 0) &
    (F.col("activity_count") > 0) &
    (F.col("event_weekday").between(0, 6))
)

# Chọn các feature cuối cùng
feature_cols = [
    "event_timestamp",
    "user_id",
    "product_id",
    "user_session",
    "price",
    "brand",
    "category_code_level1",
    "category_code_level2",
    "event_weekday",
    "activity_count",
    "is_purchased",
]
final_df = spark_df.select(*feature_cols)

# Xuất DataFrame cuối cùng ra file CSV
final_df

ModuleNotFoundError: No module named 'pyspark'

In [3]:
df.shape

(67501979, 9)

In [4]:
df.head(5)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,NaN,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2


In [5]:
df['event_time'] = pd.to_datetime(df['event_time'], format='%Y-%m-%d %H:%M:%S UTC')

In [11]:
start_date = '2019-11-28 00:00:00'
end_date = '2019-11-30 23:59:59'

filtered_df = df[(df['event_time'] >= start_date) & (df['event_time'] <= end_date)]

filtered_df.head(5)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
62234297,2019-11-28 00:00:00,view,12300394,2053013556311359947,construction.tools.drill,NaN,46.82,571535080,a7d319fe-1894-4a87-8dad-0535466d9a57
62234298,2019-11-28 00:00:00,view,28713252,2053013565639492569,apparel.shoes,respect,89.84,554617586,2e944490-59e9-4863-90dc-412e15ca3aa1
62234299,2019-11-28 00:00:00,view,1004767,2053013555631882655,electronics.smartphone,samsung,246.24,515425837,aba676e8-72de-4ea2-a4da-8eb6f3dfae7e
62234300,2019-11-28 00:00:02,view,26300105,2053013563584283495,NaN,lucente,699.89,543799816,9a89c99d-eab9-40fc-842f-0ebbd7da6106
62234301,2019-11-28 00:00:02,view,1307353,2053013558920217191,computers.notebook,acer,411.83,568271465,f2c45be2-c4cf-4caf-9c40-eaf0634b80dc


In [ ]:
filtered_df.to_csv('validated_streaming.csv', index=False)